In [1]:
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"nuthangowda","key":"3b15e291dc050f84ec9c5720fcfc2df8"}'}

In [2]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 /root/.kaggle/kaggle.json

In [3]:
!kaggle datasets download jerzydziewierz/bee-vs-wasp

 96% 538M/559M [00:04<00:00, 157MB/s]
100% 559M/559M [00:04<00:00, 123MB/s]


In [4]:
!unzip /content/bee-vs-wasp.zip

Streaming output truncated to the last 5000 lines.
  inflating: kaggle_bee_vs_wasp/other_noinsect/576513.jpg  
  inflating: kaggle_bee_vs_wasp/other_noinsect/576622.jpg  
  inflating: kaggle_bee_vs_wasp/other_noinsect/576810.jpg  
  inflating: kaggle_bee_vs_wasp/other_noinsect/576855.jpg  
  inflating: kaggle_bee_vs_wasp/other_noinsect/576993.jpg  
  inflating: kaggle_bee_vs_wasp/other_noinsect/577087.jpg  
  inflating: kaggle_bee_vs_wasp/other_noinsect/577126.jpg  
  inflating: kaggle_bee_vs_wasp/other_noinsect/577128.jpg  
  inflating: kaggle_bee_vs_wasp/other_noinsect/577283.jpg  
  inflating: kaggle_bee_vs_wasp/other_noinsect/577310.jpg  
  inflating: kaggle_bee_vs_wasp/other_noinsect/577643.jpg  
  inflating: kaggle_bee_vs_wasp/other_noinsect/577807.jpg  
  inflating: kaggle_bee_vs_wasp/other_noinsect/577847.jpg  
  inflating: kaggle_bee_vs_wasp/other_noinsect/578261.jpg  
  inflating: kaggle_bee_vs_wasp/other_noinsect/578295.jpg  
  inflating: kaggle_bee_vs_wasp/other_noinsect/57

In [5]:
import numpy as np 
import pandas as pd
import tensorflow as tf
from tensorflow.keras.preprocessing import image as img
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, MaxPooling2D, Flatten, Dropout, Activation
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import os
from random import shuffle

In [6]:
main_path = '/content/Main_path'
training_path=main_path+'/training'
testing_path = main_path+'/validation'
bee_training=training_path+'/bee'
wasp_training=training_path+'/wasp'
other_insects_training=training_path+'/other_insects'
other_noinsects_training=training_path+'/other_noinsects'
bee_testing=testing_path+'/bee'
wasp_testing=testing_path+'/wasp'
other_insects_testing=testing_path+'/other_insects'
other_noinsects_testing=testing_path+'/other_noinsects'

In [7]:
directories = [main_path, training_path, testing_path,bee_training,wasp_training, other_insects_training, other_noinsects_training, bee_testing, wasp_testing,other_insects_testing, other_noinsects_testing]

In [8]:
def directory_creation(path):
    os.mkdir(path)

In [9]:
for i in directories:
  directory_creation(i)

In [10]:
import shutil

In [11]:
def copy_files(original, dest_training, dest_testing):
  l = os.listdir(original)
  training_length = int(len(l)*0.8)
  shuffle(l)
  for i in range(len(l)):
    if i < training_length:
      shutil.copy(original+'/'+l[i],dest_training)
    else:
      shutil.copy(original+'/'+l[i], dest_testing)

In [12]:
copy_files('/content/kaggle_bee_vs_wasp/bee1', '/content/Main_path/training/bee','/content/Main_path/validation/bee')
copy_files('/content/kaggle_bee_vs_wasp/bee2', '/content/Main_path/training/bee', '/content/Main_path/validation/bee')
copy_files('/content/kaggle_bee_vs_wasp/other_insect', '/content/Main_path/training/other_insects', '/content/Main_path/validation/other_insects')
copy_files('/content/kaggle_bee_vs_wasp/other_noinsect', '/content/Main_path/training/other_noinsects', '/content/Main_path/validation/other_noinsects')
copy_files('/content/kaggle_bee_vs_wasp/wasp1','/content/Main_path/training/wasp', '/content/Main_path/validation/wasp' )
copy_files('/content/kaggle_bee_vs_wasp/wasp2','/content/Main_path/training/wasp', '/content/Main_path/validation/wasp'  )

In [13]:
datagenerator = ImageDataGenerator(rescale = 1/250., zoom_range= 0.2)

In [14]:
batch_size = 8 #accessing all our data both training and testing
training_data = datagenerator.flow_from_directory(directory = training_path,
                                                  target_size = (150,150),
                                                  batch_size = batch_size,)
testing_data = datagenerator.flow_from_directory(directory = testing_path,
                                                  target_size = (150,150),
                                                  batch_size = batch_size)

Found 9134 images belonging to 4 classes.
Found 2287 images belonging to 4 classes.


In [15]:
model = Sequential() #making our CNN
model.add(Conv2D(filters = 32, kernel_size = (3, 3), activation = 'relu', input_shape = training_data.image_shape, padding = 'same'))
model.add(MaxPooling2D(pool_size = (2, 2)))
model.add(Dropout(rate = 0.3))
model.add(Conv2D(filters = 64, kernel_size = (3, 3), activation = 'relu',padding = 'same'))
model.add(MaxPooling2D(pool_size = (2, 2)))
model.add(Dropout(rate = 0.2))
model.add(Conv2D(filters = 126, kernel_size = (3, 3), activation = 'relu', padding = 'same'))
model.add(MaxPooling2D(pool_size = (2, 2)))
model.add(Dropout(rate = 0.15))
model.add(Flatten())
model.add(Dense(units = 32, activation = 'relu'))
model.add(Dropout(rate = 0.15))
model.add(Dense(units = 64, activation = 'relu'))
model.add(Dropout(rate = 0))
model.add(Dense(units = len(set(training_data.classes)), activation = 'softmax'))
model.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics = ['accuracy'])

In [16]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 150, 150, 32)      896       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 75, 75, 32)       0         
 )                                                               
                                                                 
 dropout (Dropout)           (None, 75, 75, 32)        0         
                                                                 
 conv2d_1 (Conv2D)           (None, 75, 75, 64)        18496     
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 37, 37, 64)       0         
 2D)                                                             
                                                                 
 dropout_1 (Dropout)         (None, 37, 37, 64)        0

In [17]:
fitted_model = model.fit_generator(training_data,
                        steps_per_epoch = 250,
                        epochs = 15,
                        validation_data = testing_data,
                        validation_steps = 1000)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  """


Epoch 1/15
250/250 [==============================] - 51s 165ms/step - loss: 1.2289 - accuracy: 0.4705 - val_loss: 1.1103 - val_accuracy: 0.5426
Epoch 2/15
250/250 [==============================] - 20s 81ms/step - loss: 1.0778 - accuracy: 0.5715
Epoch 3/15
250/250 [==============================] - 21s 82ms/step - loss: 0.9936 - accuracy: 0.6045
Epoch 4/15
250/250 [==============================] - 21s 83ms/step - loss: 0.9357 - accuracy: 0.6351
Epoch 5/15
250/250 [==============================] - 20s 81ms/step - loss: 0.9374 - accuracy: 0.6341
Epoch 6/15
250/250 [==============================] - 20s 81ms/step - loss: 0.9116 - accuracy: 0.6240
Epoch 7/15
250/250 [==============================] - 20s 80ms/step - loss: 0.8888 - accuracy: 0.6326
Epoch 8/15
250/250 [==============================] - 20s 80ms/step - loss: 0.8252 - accuracy: 0.6705
Epoch 9/15
250/250 [==============================] - 20s 81ms/step - loss: 0.8448 - accuracy: 0.6517
Epoch 10/15
250/250 [==================

In [20]:
model.save('bees_vs_other.h5')

In [ ]:
from google.colab import files
files.download('/content/output') 

In [21]:
def testing_image(image_directory): #testing out our model
    test_image = img.load_img(image_directory, target_size = (150, 150))
    test_image = img.img_to_array(test_image)
    test_image = np.expand_dims(test_image, axis = 0)
    result = model.predict(x = test_image)
    print(result)
    if result[0][2]==max(result[0]):
        print('Other than insects')
    elif result[0][1]==max(result[0]):
        print('Insects')
    elif result[0][0]==max(result[0]):
        print('Bees')
    else:
        print('oooo its the wasps')

In [23]:
from IPython.display import Image
Image("../input/bee-vs-wasp/kaggle_bee_vs_wasp/bee1/10092043833_7306dfd1f0_n.jpg")